In [1]:
import pandas as pd
from yahoo_fin.stock_info import get_data
import yahoo_fin.stock_info as ys
import matplotlib.pyplot as plt


stock = "2330.TW"
df = get_data(stock)
dic = { "open":"Open" , "high":"High" , "low":"Low", "close":"Close" , "adjclose":"Adjclose","volume" : "Volume" , "ticker":"Ticler"}
df = df.rename(columns = dic)
df.index.name = "Date"
df.to_csv("data\\"+stock+".csv")
print(df.tail())

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.
              Open    High     Low   Close  Adjclose      Volume   Ticler
Date                                                                     
2024-10-24  1070.0  1075.0  1055.0  1060.0    1060.0  38236996.0  2330.TW
2024-10-25  1065.0  1070.0  1060.0  1065.0    1065.0  22009575.0  2330.TW
2024-10-28  1075.0  1080.0  1050.0  1050.0    1050.0  40142324.0  2330.TW
2024-10-29  1035.0  1040.0  1020.0  1040.0    1040.0  45415773.0  2330.TW
2024-10-30  1035.0  1055.0  1030.0  1030.0    1030.0  39095327.0  2330.TW


In [ ]:
def dataget(stock):
    df = get_data(stock)
    dic = { "open":"Open" , "high":"High" , "low":"Low", "close":"Close" , "adjclose":"Adjclose","volume" : "Volume" , "ticker":"Ticler"}
    df = df.rename(columns = dic)
    df.index.name = "Date"
    df.to_csv("data\\"+stock+".csv")
    print(df.tail())

In [1]:
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import time

#出錯出現一次
import warnings
warnings.simplefilter("once", FutureWarning)

symbol = "2330.TW"
ATR_Len = 40
HL_Len = 45
initial_stop = 2 #初幾個ATR
trailing_stop = 3 #過程幾個ATR
AMT = 2000000
rise = 1.2

#設定起始時間
start = "2023/1/1"
end = dt.date.today()
start = dt.datetime.strptime(start,"%Y/%m/%d").date()
date_range = pd.date_range(start,end)
df_temp = pd.DataFrame(index = date_range)

def tralling(rise , initial_stop , trailing_stop):
    
    symbol = "2330.TW"
    ATR_Len = 40
    HL_Len = 45
    AMT = 2000000

    #設定起始時間
    start = "2023/1/1"
    end = dt.date.today()
    start = dt.datetime.strptime(start,"%Y/%m/%d").date()
    date_range = pd.date_range(start,end)
    df_temp = pd.DataFrame(index = date_range)
    
    #取的資料
    df = get_data(symbol)
    dic = {"open":"Open","high":"High","low":"Low","close":"Close","adjclose":"Adjclose","volume":"Volume","ticker":"Ticker"}
    df = df.rename(columns = dic)
    df.index.name = "Date"
    
    #df_temp = df_temp.join(df).dropna()
    
    #基本指標
    df["周線"] = df["Close"].rolling(5).mean()
    df["月線"] = df["Close"].rolling(20).mean()
    df["季線"] = df["Close"].rolling(60).mean()
    df["HH"] = df["High"].rolling(HL_Len).max().shift(1)
    df["LL"] = df["Low"].rolling(HL_Len).min().shift(1)
    
    #移動式停損
    df["H-L"] = df["High"] - df["Low"]
    df["C1-L"] = abs(df["Close"].shift(1) - df["Low"])
    df["H-C1"] = abs(df["High"] - df["Close"].shift(1))
    df["TrueRange"] = df[["H-L","C1-L","H-C1"]].max(axis=1)
    df["ATR"] = df["TrueRange"].rolling(ATR_Len).mean()
    df["H/L"] = ( df["High"] + df["Low"] ) /2
    
    #DMI策略
    df["P"] = df["High"] - df["High"].shift(1)
    df["M"] = df["Low"].shift(1) - df["Low"]
    df["PDM"] = np.where(df["P"] > df["M"] , df["P"] , 0)
    df["MDM"] = np.where((df["M"] > 0 ) & (df["P"] < df["M"]) , df["M"] , 0)
    df["APDM"] = df["PDM"].rolling(ATR_Len).mean()
    df["AMDM"] = df["MDM"].rolling(ATR_Len).mean()
    df["pDI"] = df["APDM"] / df["ATR"] *100
    df["mDI"] = df["AMDM"] / df["ATR"] * 100
    
    #做多設定
    sign = False
    df["signals_long"] = np.zeros(np.size(df["Close"])) #歸零 ； 1 買 ，-1賣
    df["positions_long"] = np.zeros(np.size(df["Close"])) #未平倉
    df["longEntry_Price"] = np.zeros(np.size(df["Close"])) # 買入價
    #@@@@@@@@@@@@@@@@@@@@@
    df["longEntry_High"] = np.zeros(np.size(df["Close"])) #最高
    df["long_stop"] = np.zeros(np.size(df["Close"])) #停損
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    df["longExit_Price"] = np.zeros(np.size(df["Close"])) #出場
    df["longcurrentcontract"] = np.zeros(np.size(df["Close"])) #張數
    df["longCommision"] = np.zeros(np.size(df["Close"])) #手續費
    df["longCloseEquity"] = np.zeros(np.size(df["Close"])) #已平倉損益
    df["longOpenEquity"] = np.zeros(np.size(df["Close"])) #未平倉損益
    df["longCumsum_OpenEquit"] = np.zeros(np.size(df["Close"])) #累計未平倉損益
    df["longCumsum_CloseEquit"] = np.zeros(np.size(df["Close"])) #累計已平倉損益
    
    #資料載入
    df_temp = df_temp.join(df).dropna()
    df = df_temp
    #回測
    start = dt.datetime.now()
    for i in range(2 , df["Close"].size):
        time.sleep(0.1)
        if sign :
            df.loc[df.index[i],"longEntry_Price"] = df.loc[df.index[i-1],"longEntry_Price"]
            df.loc[df.index[i],"positions_long"] = df.loc[df.index[i-1],"positions_long"]
            #移動式停損
            if df["High"].iloc[i] > df["High"].iloc[i-1]:
                df.loc[df.index[i],"longEntry_High"] = df["High"].iloc[i]
            else:
                df.loc[df.index[i] , "longEntry_High"] = df.loc[df.index[i-1] , "longEntry_High"]
            if df["Close"].iloc[i] > df["longEntry_High"].iloc[i-1]:
                df.loc[df.index[i] , "long_stop"] = min(df["Low"].iloc[i],df["Low"].iloc[i-1]) - trailing_stop*df["ATR"].iloc[i]
            else:
                df.loc[df.index[i],"long_stop"] = df["long_stop"].iloc[i-1]
        if not sign:
            df.loc[df.index[i] , "long_stop"] = np.nan #為了畫圖用
            #看昨天有沒有訊號，有的話今天下單
            if df["pDI"].iloc[i-1] > df["mDI"].iloc[i-1] +10:
                df.loc[df.index[i] , "signals_long"] = 1 #訊號
                df.loc[df.index[i] , "positions_long"] = 1 #部位
                df.loc[df.index[i] , "longEntry_Price"] = df["H/L"].iloc[i] #價格
                #停損
                df.loc[df.index[i] , "longEntry_High"] = df["High"].iloc[i]
                df.loc[df.index[i] , "long_stop"] = min(df["Low"].iloc[i],df["Low"].iloc[i-1])- initial_stop*df["ATR"].iloc[i]
                #大訊號
                sign = True
        elif sign:
            if df["Close"].iloc[i-1] < df["long_stop"].iloc[i-2] or df["Close"].iloc[i-1] > df["longEntry_Price"].iloc[i-1] * rise:
                df.loc[df.index[i] , "signals_long"] = -1
                df.loc[df.index[i] , "longExit_Price"] = df["H/L"].iloc[i]
                sign = False
    df["positions_long"] = df["signals_long"].cumsum()
    df.loc[(df.longEntry_Price != 0),"longcurrentcontract"] = round (AMT/(df.longEntry_Price*1000)) #部位
    
    LE = (df.positions_long == 1) & (df.signals_long == 1) #當天購買部位
    LX = (df.positions_long.shift(1) == 1) & (df.signals_long == -1) #平倉前一天
    
    df["longCommission"]=df.longExit_Price*df.longcurrentcontract.shift(1)*1000*0.003#計算交易成本
    
    df.loc[LE,"longOpenEquity"]=(df.Close-df.longEntry_Price)*df.longcurrentcontract*1000 #進場當天是收盤-進場價
    df.loc[(df.positions_long==1)&(df.signals_long!=1),"longOpenEquity"]=(df.Close-df.Close.shift(1))*df.longcurrentcontract*1000 #進場後是今收-昨收
    df.loc[LX,"longOpenEquity"]=(df.longExit_Price-df.Close.shift(1))*df.longcurrentcontract.shift(1)*1000-df.longCommission #出場當天是出場價-昨收
    
    df.loc[LX,"longCloseEquity"]=(df.longExit_Price-df.longEntry_Price.shift(1))*df.longcurrentcontract.shift(1)*1000-df.longCommission
    
    df["longCumsum_OpenEquity"]=df["longOpenEquity"].cumsum()+AMT
    df["longCumsum_CloseEquity"]=df["longCloseEquity"].cumsum()+AMT
    
    print(dt.datetime.now()-start)
    
    return df
print("Done")

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.
Done


In [2]:
start = "2023/1/1"
end = dt.date.today()

indexlist=[]#空的list 等下計算出每個參數的ColumnName要append進來
performancelist=[]#空的list 等下計算出每個參數的最後一天的績效值要append進來
sharplist=[]#空的list 等下計算出每個參數的Sharp值要append進來
df_empty=pd.DataFrame(index=pd.date_range(start,end))

for rise in range(10,15):
    #print("Run_HL_len=",HL_len)
    for initial_stop in range(1,4,1):
        #print("Run_initial_stop=",initial_stop)
        for trailing_stop in range(1,4,1):
            #print("Run_trailing_stop=",trailing_stop)
            df=tralling(rise/10,initial_stop,trailing_stop)
            ColumnName=str(rise/10)+","+str(initial_stop)+","+str(trailing_stop)
            indexlist.append(ColumnName)#========================>ColumnName is str
            df_openequity=df[["longCumsum_OpenEquity"]].rename(columns={"longCumsum_OpenEquity":ColumnName})
            performancelist.append(df_openequity.iloc[-1,0])#===============>performance is value
            df_daily_R=df[["longCumsum_OpenEquity"]].pct_change().dropna()
            sharp=(df_daily_R.mean()/df_daily_R.std())*252**0.5
            sharplist.append(sharp.iloc[0])#=================================>sharp is value
            #df.plot(title=symbol,figsize=(15,5),grid=True)
            #plt.show()
            df_=df_empty.join(df_openequity).dropna()
df_openequity=pd.DataFrame({"openequity":performancelist},index=indexlist)
df_sharp=pd.DataFrame({"sharp":sharplist},index=indexlist)
df_merge=df_openequity.join(df_sharp)
df_merge_sort=df_merge.sort_values(by=["openequity","sharp"],ascending=False)
df_merge_sort.to_csv("C://download//performance//"+symbol+"optimize.csv")
list_optimize=df_merge_sort.index
print(symbol+"optimization=",list_optimize[0])
after_optimize_HL_len=int(list_optimize[0].split(",")[0])
after_optimize_initial=int(list_optimize[0].split(",")[1])
after_optimize_trailing=int(list_optimize[0].split(",")[2])

0:00:50.961393


KeyboardInterrupt: 